In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310114  [   64/60000]
loss: 2.305957  [ 6464/60000]
loss: 2.284858  [12864/60000]
loss: 2.279474  [19264/60000]
loss: 2.270754  [25664/60000]
loss: 2.235430  [32064/60000]
loss: 2.247225  [38464/60000]
loss: 2.214430  [44864/60000]
loss: 2.214965  [51264/60000]
loss: 2.187000  [57664/60000]
Test Error: 
 Accuracy: 33.2%, Avg loss: 2.181375 

Epoch 2
-------------------------------
loss: 2.194663  [   64/60000]
loss: 2.187042  [ 6464/60000]
loss: 2.133004  [12864/60000]
loss: 2.146550  [19264/60000]
loss: 2.108569  [25664/60000]
loss: 2.054172  [32064/60000]
loss: 2.082226  [38464/60000]
loss: 2.013861  [44864/60000]
loss: 2.020331  [51264/60000]
loss: 1.953157  [57664/60000]
Test Error: 
 Accuracy: 56.1%, Avg loss: 1.949039 

Epoch 3
-------------------------------
loss: 1.987581  [   64/60000]
loss: 1.956875  [ 6464/60000]
loss: 1.845724  [12864/60000]
loss: 1.877912  [19264/60000]
loss: 1.780034  [25664/60000]
loss: 1.729826  [32064/600